In [126]:
import os
from functools import partial
from typing import Annotated
from typing import TypedDict

from dotenv import find_dotenv
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage
from langchain_core.messages import AIMessage
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

In [127]:
load_dotenv(find_dotenv())


class State(TypedDict):
    messages: Annotated[list, add_messages]
    sender: str

model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY"),
    temperature=0.4,        
)
memory = MemorySaver()

In [128]:
def create_agent(llm, system_message: str, tools=None, **kwargs):
    """Create an agent."""
    prompt = ChatPromptTemplate.from_messages(
        [system_message, MessagesPlaceholder(variable_name="messages"),]
    )
    prompt = prompt.partial(**kwargs)
    if tools is not None:
        return prompt | llm.bind_tools(tools)
    return prompt | llm

def agent_node(state, agent, name):
    result = agent.invoke(state)
    result = AIMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        "sender": name,
    }

In [129]:
system_message = """
You are an assistant responsible for extracting a class from Java code. You will be provided with two Java files: a "model solution" and a "submitted solution."
Your job is to extract the Java class from the files and provide a JSON output with the following structure.

```json
{{
    "model": {{
        "imports": "...",
        "class": "..."
    }},
    "submission": {{
        "imports": "...",
        "class": "..."
    }}
}}
```
"""

class_extractor_agent = create_agent(model, system_message)
class_extractor_node = partial(agent_node, agent=class_extractor_agent, name="class-extractor")

In [130]:
system_message = """
You are an assistant responsible for extracting rubric details from the input JSON data. Provide your output in JSON as per the rubric.
Do not assign any marks to each rubric item. You must only extract details about the input classes with respect to each rubric item and provide that as a comment.
Return your data in JSON format. Note that the "<rubric>" entry in the JSON needs to be changed as per each rubric item given to you.

RUBRIC
{rubric}

OUTPUT FORMAT
```json
{{
    "model": {{
        "<rubric>": "...",
        ...
    }},
    "submission": {{
        "<rubric>": "...",
        ...
    }}
}}
```
"""
with open("data/simple-scenario/rubric.md", "r") as f:
    rubric = f.read()

rubric_extractor_agent = create_agent(model, system_message, rubric=rubric)
rubric_extractor_node = partial(agent_node, agent=rubric_extractor_agent, name="rubric-extractor")

In [131]:
system_message = """
You are an assistant responsible for evaluating the input Java code based on the rubric provided. Assign a numeric score based on the comments provided to you. Note that these marks are awarded step-by-step, hence you must provide marks per step and highlight each step as well.
Along with the numeric scores (as per the below rubric), also include detailed comments about the correctness, errors and suggestions for improvement in the submitted code.
Return your response as JSON data. Do not return anything else other than the JSON.

RUBRIC
{rubric}

OUTPUT FORMAT
```json
{{
    "model": {{
        "<rubric>": {{
            "marks": [1, 2, 3],
            "comments": ["..."]
        }}
    }},
    "submission": {{
        "<rubric>": {{
            "marks": [1, 2, 3],
            "comments": ["..."]
        }}
    }}
}}
```
"""

initial_eval_agent = create_agent(model, system_message, rubric=rubric)
initial_eval_node = partial(agent_node, agent=initial_eval_agent, name="initial-evaluation")

In [132]:
system_message = """
You are an assistant responsible for verifying the evaluation results provided to you as JSON. Verify that the evaluation results are appropriate, and if they are not, update the marks and comments where necessary.
Return the updated marks and comments in JSON format. It is possible that there are no changes in the marks/comments. You must only update the marks and comments, and not remove anything.
Do not return anything else other than the JSON.

OUTPUT FORMAT
```json
{{
    "model": {{
        "<rubric>": {{
            "marks": [1, 2, 3],
            "comments": ["..."]
        }}
    }},
    "submission": {{
        "<rubric>": {{
            "marks": [1, 2, 3],
            "comments": ["..."]
        }}
    }}
}}
```
"""

review_eval_agent = create_agent(model, system_message)
review_eval_node = partial(agent_node, agent=review_eval_agent, name="review-evaluation")

In [140]:
system_message = """
You are an assistant responsible for extracting the marks key from the input JSON data. Extract the marks as a comma-separated list. Return the data in the below JSON format.

OUTPUT FORMAT
```json
{{
    "model": [1, 2, 3],
    "submission": [1, 2, 3]
}}
```
"""

extract_marks_agent = create_agent(model, system_message)
extract_marks_node = partial(agent_node, agent=extract_marks_agent, name="extract-marks")

In [134]:
@tool
def sum_marks(marks: list[int]) -> int:
    """Add a list of marks

    Args:
        a (list(int)): list of marks

    Returns:
        int: Result of sum(marks)
    """
    return sum(marks)

system_message = """
You are an assistant responsible for calculating the sum of marks provided to you. You must use the 'sum_marks' tool to assist with this process.

OUTPUT FORMAT
```json
{{
    "model": 1,
    "submission": 1
}}
```
"""

tools = [sum_marks]
total_marks_agent = create_agent(model, system_message, tools)
total_marks_node = partial(agent_node, agent=total_marks_agent, name="total-marks")
tool_node = ToolNode(tools)

Key 'title' is not supported in schema, ignoring
Key 'title' is not supported in schema, ignoring


In [135]:
def router(state):
    sender = state["sender"]
    if sender == "class-extractor":
        return "rubric-extractor"
    elif sender == "rubric-extractor":
        return "initial-evaluation"
    elif sender == "initial-evaluation":
        return "review-evaluation"
    elif sender == "review-evaluation":
        return "extract-marks"
    elif sender == "extract-marks":
        return "total-marks"
    elif sender == "total-marks":
        messages = state["messages"]
        last_message = messages[-1]
        if last_message.tool_calls:
            return "call_tool"
    return END

In [141]:
workflow = StateGraph(State)

workflow.add_node("class-extractor", class_extractor_node)
workflow.add_node("rubric-extractor", rubric_extractor_node)
workflow.add_node("initial-evaluation", initial_eval_node)
workflow.add_node("review-evaluation", review_eval_node)
workflow.add_node("extract-marks", extract_marks_node)
workflow.add_node("total-marks", total_marks_node)
workflow.add_node("call_tool", tool_node)
workflow.add_edge("class-extractor", "rubric-extractor")
workflow.add_edge("rubric-extractor", "initial-evaluation")
workflow.add_edge("initial-evaluation", "review-evaluation")
workflow.add_edge("review-evaluation", "extract-marks")
workflow.add_edge("extract-marks", "total-marks")
workflow.add_conditional_edges(
    "total-marks",
    router,
    {"call_tool": "call_tool", END: END},
)
workflow.add_conditional_edges(
    "call_tool",
    # Each agent node updates the 'sender' field
    # the tool calling node does not, meaning
    # this edge will route back to the original agent
    # who invoked the tool
    lambda x: x["sender"],
    {
        "total-marks": "total-marks",
    },
)
workflow.set_entry_point("class-extractor")
graph = workflow.compile(checkpointer=memory)

In [137]:
graph.get_graph().draw_png("graph.png")

In [138]:
async def stream_graph_updates(user_input: str):
    messages = {"messages": [HumanMessage(content=user_input)]}
    config = {"configurable": {"thread_id": 1}}
    async for output in graph.astream(messages, stream_mode="updates", config=config):
        for value in output.values():
            print("Assistant:", value["messages"][-1].pretty_print())
        print("\n---\n")

In [142]:
with open("data/simple-scenario/model_solution.md", "r") as f:
    model_solution = f.read()

with open("data/simple-scenario/student_solution.md", "r") as f:
    student_solution = f.read()

user_input = f"""
Model Solution
{model_solution}
---
Student Solution
{student_solution}
"""
await stream_graph_updates(user_input)

================================== Ai Message ==================================
Name: class-extractor

```json
{
    "model": {
        "imports": "import java.util.Scanner;",
        "class": "public class StringManipulator {\n    public static void main(String[] args) {\n        Scanner sc = new Scanner(System.in);\n        System.out.print(\"Enter a string: \");\n        String input = sc.nextLine();\n        System.out.println(\"Original String: \" + input);\n        System.out.println(\"Uppercase String: \" + input.toUpperCase());\n        String reversed = new StringBuilder(input).reverse().toString();\n        System.out.println(\"Reversed String: \" + reversed);\n        System.out.println(\"Number of Characters: \" + input.length());\n        sc.close();\n    }\n}"
    },
    "submission": {
        "imports": "import java.util.Scanner;",
        "class": "public class StringManipulator {\n    public static void main(String[] args) {\n        Scanner sc = new Scanner(System.i

Gemini produced an empty response. Continuing with empty message
Feedback: 


================================== Ai Message ==================================
Name: rubric-extractor
Assistant: None

---

================================== Ai Message ==================================
Name: initial-evaluation

```json
{
    "model": {
        "compilation_and_execution": {
            "marks": [5, 5, 5],
            "comments": ["The program compiles and runs without any errors."]
        },
        "user_input_handling": {
            "marks": [5, 5, 5],
            "comments": ["The program correctly prompts the user to enter a string and reads the input using `sc.nextLine()`."]
        },
        "displaying_original_string": {
            "marks": [5, 5, 5],
            "comments": ["The program displays the original string with appropriate labeling."]
        },
        "converting_to_uppercase": {
            "marks": [5, 5, 5],
            "comments": ["The program correctly converts the string to uppercase using `toUpperCase()` and displays it with approp

Gemini produced an empty response. Continuing with empty message
Feedback: 


================================== Ai Message ==================================
Name: review-evaluation
Assistant: None

---

================================== Ai Message ==================================
Name: extract-marks

```json
{
    "model": [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    "submission": [1, 2, 2, 1, 2, 2, 2, 2, 3, 1, 1, 2, 5, 5, 5]
}
```
Assistant: None

---

{'name': 'sum_marks', 'description': 'Add a list of marks\n\n    Args:\n        a (list(int)): list of marks\n\n    Returns:\n        int: Result of sum(marks)', 'parameters': {'type_': 6, 'description': 'Add a list of marks\n\nArgs:\n    a (list(int)): list of marks\n\nReturns:\n    int: Result of sum(marks)', 'properties': {'marks': {'type_': 5, 'items': {'type_': 3, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 'properties': {}, 'required': []}, 'format_': '', 'description': '', 'nullable': False, 'enum': [], 'max_items': '0', 'min_items': '0', 

Gemini produced an empty response. Continuing with empty message
Feedback: 


================================== Ai Message ==================================
Name: total-marks
Assistant: None

---

